# Image classifier

Create directory `data/dogs_vs_cats` and from <https://www.kaggle.com/c/dogs-vs-cats/data> download `train.zip` into `data/dogs_vs_cats`. 

Now we are going to uzip it and create two separate subdirectories `cats` nad `dogs` and put there correspondig images. For that you can run:

```
cd data/dogs_vs_cats
unzip train.zip
rm trian.zip
cd train
mkdir dogs
find . -name 'dog.*' -exec mv {} dogs \;
mkdir cats
find . -name 'cat.*' -exec mv {} cats \;
```

Note that we are using `find` to move since using `mv` gives `argument list too long` error (it may take some time).

In [1]:
from keras.preprocessing.image import ImageDataGenerator

image_directory = 'data/dogs_vs_cats/train'
target_width, target_height = 128, 128
batch_size = 32

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        image_directory,
        target_size=(target_width, target_height),
        batch_size=batch_size,
        class_mode='binary',
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        image_directory,
        target_size=(target_width, target_height),
        batch_size=batch_size,
        class_mode='binary',
        subset='validation')

/Users/bartek.skorulski/Envs/py3.6/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [8]:
len(train_generator)

625

## CNN in Keras

Convolution Layer: <https://keras.io/layers/convolutional/>
Pooling: <https://keras.io/layers/pooling/>
Flatten: <https://keras.io/layers/core/#flatten>

In [2]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense
from keras.layers import Dropout

clf = Sequential()
clf.add(Convolution2D(32, (3, 3), input_shape=(128, 128, 3), padding="same", activation="relu")) # 128x128x32
clf.add(MaxPooling2D(pool_size=(2, 2))) # 64x64x32
clf.add(Convolution2D(32, (3, 3), padding="same", activation="relu")) # 64x64x32
clf.add(MaxPooling2D(pool_size=(2, 2))) # 32x32x32
clf.add(Convolution2D(64, (3, 3), padding="same", activation="relu")) # 32x32x64
clf.add(MaxPooling2D(pool_size=(2, 2)))  # 16x16x64
clf.add(Flatten()) # 16384
clf.add(Dense(units=128, activation='relu')) #1 28
clf.add(Dropout(0.5)) # 128
clf.add(Dense(units=64, activation='relu')) # 64
clf.add(Dropout(0.5)) # 64
clf.add(Dense(units=1, activation='sigmoid')) # 1

In [3]:
for i in range(len(clf.layers)):
    print("layer ", i, ":")
    print(clf.layers[i].name)
    print(clf.layers[i].output)

layer  0 :
conv2d_1
Tensor("conv2d_1/Relu:0", shape=(?, 128, 128, 32), dtype=float32)
layer  1 :
max_pooling2d_1
Tensor("max_pooling2d_1/MaxPool:0", shape=(?, 64, 64, 32), dtype=float32)
layer  2 :
conv2d_2
Tensor("conv2d_2/Relu:0", shape=(?, 64, 64, 32), dtype=float32)
layer  3 :
max_pooling2d_2
Tensor("max_pooling2d_2/MaxPool:0", shape=(?, 32, 32, 32), dtype=float32)
layer  4 :
conv2d_3
Tensor("conv2d_3/Relu:0", shape=(?, 32, 32, 64), dtype=float32)
layer  5 :
max_pooling2d_3
Tensor("max_pooling2d_3/MaxPool:0", shape=(?, 16, 16, 64), dtype=float32)
layer  6 :
flatten_1
Tensor("flatten_1/Reshape:0", shape=(?, ?), dtype=float32)
layer  7 :
dense_1
Tensor("dense_1/Relu:0", shape=(?, 128), dtype=float32)
layer  8 :
dropout_1
Tensor("dropout_1/cond/Merge:0", shape=(?, 128), dtype=float32)
layer  9 :
dense_2
Tensor("dense_2/Relu:0", shape=(?, 64), dtype=float32)
layer  10 :
dropout_2
Tensor("dropout_2/cond/Merge:0", shape=(?, 64), dtype=float32)
layer  11 :
dense_3
Tensor("dense_3/Sigmoid:

In [4]:
clf.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])

In [7]:
clf.fit_generator(
        train_generator,
        steps_per_epoch=250,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=50)

Epoch 1/20
250/250 [==============================] - 125s 501ms/step - loss: 0.6609 - acc: 0.6129 - val_loss: 0.6243 - val_acc: 0.6619
Epoch 2/20
250/250 [==============================] - 445s 2s/step - loss: 0.6154 - acc: 0.6625 - val_loss: 0.5684 - val_acc: 0.7170
Epoch 3/20
250/250 [==============================] - 132s 526ms/step - loss: 0.5778 - acc: 0.7031 - val_loss: 0.5374 - val_acc: 0.7250
Epoch 4/20
250/250 [==============================] - 131s 524ms/step - loss: 0.5645 - acc: 0.7181 - val_loss: 0.5096 - val_acc: 0.7475
Epoch 5/20
250/250 [==============================] - 127s 506ms/step - loss: 0.5306 - acc: 0.7416 - val_loss: 0.5160 - val_acc: 0.7303
Epoch 6/20
250/250 [==============================] - 125s 498ms/step - loss: 0.5080 - acc: 0.7556 - val_loss: 0.4925 - val_acc: 0.7725
Epoch 7/20
250/250 [==============================] - 129s 515ms/step - loss: 0.4970 - acc: 0.7695 - val_loss: 0.4603 - val_acc: 0.7900
Epoch 8/20
250/250 [==============================]